Modèle Stat ARIMA... (SARIMA)

librairie : statmodel (library, package ARIMA)


Model LLM, Ollama, TimeCHATGPT...

à compléter et battre le BTC tout seul est quasiment impossible 

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, LSTM
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.layers import Bidirectional

from modules.utils import feature_engineering_last, filter_data_by_year_month

from sklearn.model_selection import train_test_split

In [ ]:
X = pd.read_csv(
    "/Users/elouan/Repo Github ElouanBahri/Predicting_crypto_prices/Historical Prices for BTCUSDT"
)

In [ ]:
YEARS = [2020,2021,2022,2023,2024,2025]

Data = filter_data_by_year_month(X, YEARS)

df = feature_engineering_last(X)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute correlation matrix
corr_matrix = df.corr()

# Plot heatmap
plt.figure(figsize=(12,8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.show()


In [ ]:
print(df['target'].value_counts(normalize=True))


In [ ]:
# Check for NaN values in the entire DataFrame
print(df.isnull().sum())

In [ ]:
df.head(10)

In [ ]:
# df = df.replace([np.inf, -np.inf], np.nan)  # Replace inf with NaN
# df = df.fillna(df.mean())

X = df.drop(columns=["target"]).values  # Features
y = df["target"].values  # Target variable


# Reshape the data into sequences (timesteps)
timesteps = 5  # Number of timesteps for the RNN
X_sequences = []
y_sequences = []

for i in range(len(X) - timesteps):
    X_sequences.append(X[i : i + timesteps])
    y_sequences.append(y[i + timesteps])

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

split_index = int(len(X_sequences) * 0.8)  # 80% train, 20% test
X_train, X_test = X_sequences[:split_index], X_sequences[split_index:]
y_train, y_test = y_sequences[:split_index], y_sequences[split_index:]

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
print(np.isnan(X).any(), np.isnan(y).any())  # Check for NaN
print(np.isinf(X).any(), np.isinf(y).any())

In [ ]:
# dataset = tf.data.Dataset.from_tensor_slices((X1, y))
# # Calculate sizes for train and validation sets
# dataset_size = len(X1)  # Total number of samples in your dataset
# train_size = int(0.8 * dataset_size)

# # Shuffle the dataset if needed
# dataset = dataset.shuffle(buffer_size=dataset_size)

# # Split the dataset
# train_dataset = dataset.take(train_size)       # Take the first 80%
# val_dataset = dataset.skip(train_size)         # Skip the first 80%, take the remaining 20%

# # Batch both datasets if needed
# batch_size = 32
# train_dataset = train_dataset.batch(batch_size)
# val_dataset = val_dataset.batch(batch_size)

In [ ]:
tf.random.set_seed(1234)  # for consistent results

# Simplified GRU Model
model1 = Sequential(
    [
        # Single GRU Layer
        GRU(64, input_shape=(10, X_sequences.shape[2]), name="GRU_Layer"),
        Dropout(0.2, name="Dropout_Layer"),  # Regularization
        # Fully Connected Dense Layer
        Dense(32, activation="relu", name="Dense_Layer"),  # Hidden dense layer
        # Output Layer for Binary Classification
        Dense(1, activation="sigmoid", name="Output_Layer"),  # Predict binary output
    ]
)

In [ ]:
tf.random.set_seed(1234)  # for consistent results



model2 = Sequential([
    Bidirectional(LSTM(128, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

In [ ]:
tf.random.set_seed(1234)  # for consistent results

model3 = Sequential([
    Bidirectional(LSTM(256, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

In [ ]:
model3.summary()

In [ ]:
# model = tf.keras.models.load_model("../models/")

In [ ]:
# Compile the model
model3.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(),  # 🔥 Pour classification binaire
    metrics=['accuracy'],
)

# Learning rate reduction callback
reduce_lr = ReduceLROnPlateau(
    monitor="val_accuracy",
    factor=0.1,  # Reduce LR by 90% instead of 50%
    patience=3,
    min_lr=1e-6,  # Set a minimum learning rate
)

# Model checkpoint callback
checkpoint = ModelCheckpoint(
    filepath="../models/RNN_LTSM_model3.keras",  # Save the model to this path
    monitor='val_accuracy',                 # Monitor validation accuracy
    mode='max',                        # Save when accuracy is maximized
    save_best_only=True,  # Save only if the model improves
    verbose=1,  # Display a message when saving
)
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train the model
history = model3.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,  # Increase epochs for better training
    batch_size=32,
    callbacks=[reduce_lr, checkpoint, early_stop],  # Add early stopping
)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Prédictions sur le test set
y_pred_prob = model2.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convertir en 0 ou 1

# Calculer l’accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Afficher un rapport détaillé (Precision, Recall, F1-score)
print(classification_report(y_test, y_pred))

# Matrice de confusion
import seaborn as sns
import matplotlib.pyplot as plt

conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()



In [ ]:
import matplotlib.pyplot as plt
df_test = pd.DataFrame({'Actual Return': y_test, 'Predicted': y_pred.flatten()})
df_test['Signal'] = df_test['Predicted'].shift(1)  # Décalage pour éviter la fuite d’info

# Appliquer les signaux pour simuler une stratégie
df_test['Strategy Return'] = df_test['Signal'] * df_test['Actual Return']
df_test['Cumulative Return'] = df_test['Strategy Return'].cumsum()

# Afficher la performance de la stratégie
plt.figure(figsize=(12,6))
plt.plot(df_test['Cumulative Return'], label="LSTM Trading Strategy", color='green')
plt.title("Backtest: Cumulative Returns of LSTM Strategy")
plt.legend()
plt.show()



------------------

RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train_2D = X_train.reshape(X_train.shape[0], -1)  # Flatten
X_test_2D = X_test.reshape(X_test.shape[0], -1)    # Flatten

# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_2D, y_train)

# Predict & Evaluate
y_pred_rf = rf.predict(X_test_2D)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")

-------------

XGboost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Train an XGBoost classifier
xgb = XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, random_state=42)
xgb.fit(X_train_2D, y_train)

# Make predictions
y_pred_xgb = xgb.predict(X_test_2D)

# Evaluate accuracy
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Accuracy: {accuracy_xgb:.4f}")


Feature importance

In [ ]:
import matplotlib.pyplot as plt

# Get feature importance
importance = xgb.feature_importances_

# Plot feature importance
plt.figure(figsize=(12,6))
plt.bar(range(len(importance)), importance)
plt.title("XGBoost Feature Importance")
plt.xlabel("Feature Index")
plt.ylabel("Importance Score")
plt.show()
